#Random Forest Model Predicting Overall Persuadability of Voters

For this model, I will be focusing on people who moved toward the Republican candidate between waves. The predictor variable I will be using is MOVED_AR.

##Load libraries and data set

In [1]:
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

data = pd.read_csv('FX_indicators_2020.csv', index_col='VOTER_ID')

data.head(5)

,SET_NO,OPP_SEX,AGE,HH_ND,HH_NR,HH_NI,MED_AGE,MED_AGE_M,MED_AGE_F,NH_WHITE,...,MOVED_RDMA,MOVED_DRMA,MOVED_AWMA,MOVED_ADMA,MOVED_ARMA,MOVED_RDMB,MOVED_DRMB,MOVED_AWMB,MOVED_ADMB,MOVED_ARMB
VOTER_ID,,,,,,,,,,,,,,,,,,,,,
84508,3,0,25,4,0,0,38,39,38,76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608312,1,0,35,0,0,3,46,46,46,88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
222821,3,0,73,3,0,0,42,36,48,45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137882,2,0,54,1,1,0,37,34,51,61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
531303,2,0,51,0,3,0,46,46,46,88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##Data Wrangling

In [2]:
#create a model type for csv output file
model_type ='RFmove'

#create a list of dependent variables
dv_list = ['CAND1S','CAND2S','MESSAGE','VG_14_DV','D2','R2','D3','R3','I3','CAND1_UND','CAND1_SD2','CAND1_SDA','CAND1_LD2',
           'CAND1_LDA', 'CAND1_SR2','CAND1_SRA','CAND1_LRA','CAND1_LR2','CAND1_SFT','CAND2_UND','CAND2_SD2','CAND2_SDA',
           'CAND2_LD2','CAND2_LDA', 'CAND2_SR2','CAND2_SRA','CAND2_LRA','CAND2_LR2','CAND2_SFT','MOVED_RD','MOVED_DR',
           'MOVED_AW','MOVED_U','MOVED_AD', 'MOVED_AR','MOVED_RDMA','MOVED_DRMA','MOVED_AWMA','MOVED_ADMA','MOVED_ARMA',
           'MOVED_RDMB','MOVED_DRMB','MOVED_AWMB','MOVED_ADMB','MOVED_ARMB']

#identify the dependent variable for this model
dv = 'MOVED_AR'

#create a drop variable and remove the dependent variable from the list
fields_to_drop = dv_list
fields_to_drop.remove(dv)

#remove all unneeded fields
data.drop(fields_to_drop, axis=1, inplace=True)

data.head(5)

,SET_NO,OPP_SEX,AGE,HH_ND,HH_NR,HH_NI,MED_AGE,MED_AGE_M,MED_AGE_F,NH_WHITE,...,DOITYOURSE,FINANCIALM,RELIGIOUSC,POLITICALC,MEDIANEDUC,PRS16_PD,PRS16_PR,MSG_A,MSG_B,MOVED_AR
VOTER_ID,,,,,,,,,,,,,,,,,,,,,
84508,3,0,25,4,0,0,38,39,38,76,...,0,0,0,0,12,48.8,49.6,0,0,NaN
608312,1,0,35,0,0,3,46,46,46,88,...,0,0,1,1,16,48.4,49.9,0,0,NaN
222821,3,0,73,3,0,0,42,36,48,45,...,0,0,1,1,12,47.8,50.5,0,0,NaN
137882,2,0,54,1,1,0,37,34,51,61,...,0,0,0,0,12,54.9,43.4,1,0,NaN
531303,2,0,51,0,3,0,46,46,46,88,...,0,0,1,1,15,48.3,50.1,0,0,NaN


In [3]:
#change all Y/N to 1/0 for target variable 
data[dv].replace({'N':0, 'Y':1}, inplace=True)

#drop null records from target variable and save to new dataset 
data_all = data.dropna(subset=[dv]) 
data_all.head(5) 

,SET_NO,OPP_SEX,AGE,HH_ND,HH_NR,HH_NI,MED_AGE,MED_AGE_M,MED_AGE_F,NH_WHITE,...,DOITYOURSE,FINANCIALM,RELIGIOUSC,POLITICALC,MEDIANEDUC,PRS16_PD,PRS16_PR,MSG_A,MSG_B,MOVED_AR
VOTER_ID,,,,,,,,,,,,,,,,,,,,,
404213,1,29,78,0,0,2,39,26,50,37,...,0,0,0,0,12,43.4,55.0,0,1,0.0
136981,2,0,29,5,0,0,50,49,52,82,...,0,0,1,0,12,45.6,52.7,0,0,0.0
532172,1,0,26,2,0,2,37,35,38,87,...,0,0,0,0,12,49.2,49.1,0,1,0.0
208471,1,0,57,1,2,1,37,37,38,54,...,0,0,0,0,12,45.2,53.2,0,1,0.0
261184,3,0,50,0,0,2,38,38,39,83,...,0,0,0,0,12,39.5,58.9,0,0,1.0


In [4]:
#create a train/test split using set 1 and 2 for training and set 3 for testing 
df_train = data_all[(data_all['SET_NO'] == 1) | (data_all['SET_NO'] == 2)] 
df_train.drop('SET_NO', axis=1, inplace=True) # Remove `SET_NO` as a indicator 
y_train = df_train.pop(dv).values
X_train = df_train 
df_test = data_all[data_all['SET_NO'] == 3] 
df_test.drop('SET_NO', axis=1, inplace=True) # Remove `SET_NO` as a indicator 
y_test = df_test.pop(dv).values 
X_test = df_test 

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


##Build/Evaluate/Score Model

In [5]:
#create and fit the random forest model to the dataset
rf = RandomForestClassifier(n_estimators=100, random_state=1)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

In [6]:
#evaluate the model
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.92      0.99      0.95     12666
         1.0       0.45      0.10      0.16      1168

    accuracy                           0.91     13834
   macro avg       0.69      0.54      0.56     13834
weighted avg       0.88      0.91      0.89     13834



In [7]:
# Remove extraneous fields (the DV and `SET_NO`) before scoring.
data.drop([dv, 'SET_NO'], axis=1, inplace=True)

#Decision Tree scoring
# Score the whole file.
pred = rf.predict_proba(data.values)[:, 1]
scores = pd.DataFrame({dv: pred}, index=data.index)

#export the file to csv
filename = 'RFmove.csv'.format(dv, model_type)
scores.to_csv(filename)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
